<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/736_RGOv2_StoreLevelStockouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Store-Level Stockout Attribution

---

# 1️⃣ What This Module Does — In Real Terms

This function determines:

> Of the weeks where this customer underperformed, how many coincided with a store stockout?

It calculates:

* Total gap weeks
* Number of those weeks overlapping stockouts
* Percent overlap
* A binary operational impact flag

This is crucial.

Without this logic, every revenue drop would look like customer behavior.

This module introduces operational causality.

---

# 2️⃣ Why This Is Strategically Important

Revenue decline can originate from:

* Customer disengagement
* Price sensitivity
* Market demand
* Structural erosion
* Operational failure

This module introduces cross-functional accountability.

If stockouts align with revenue gaps, the narrative shifts from:

> “Customer is churning.”

to

> “Operations constrained revenue.”

That changes intervention strategy entirely.

This is business-critical differentiation.

---

# 3️⃣ Gap Week Identification Logic

```python
if spend < baseline_revenue or spend == 0:
```

This is a clear and deterministic definition.

You define a “gap week” as:

* Any week below baseline
* Or explicitly zero spend

This is appropriate.

It aligns with your structural framework.

However, note:

You are evaluating all weeks in `sales_sorted`, not just the recent structural window.

This is acceptable for MVP — but worth noting.

---

# 4️⃣ Efficient Overlap Calculation

```python
overlap = sum(1 for (s, w) in gap_weeks if (s, w) in stockout_store_weeks)
```

This is elegant.

Because:

* `stockout_store_weeks` is a set
* Membership check is O(1)
* No scanning of stock table
* No nested loops

This is computationally efficient and clean.

That’s good engineering discipline.

---

# 5️⃣ Percent Attribution

```python
percent = overlap / total_gap_weeks
```

This gives proportional accountability.

It avoids:

* Blaming operations for 1 coincidental week
* Ignoring consistent operational friction

This creates gradation.

Not binary blame.

That’s thoughtful modeling.

---

# 6️⃣ The Impact Flag

```python
stockout_impact_flag = overlap >= 1
```

This is a simple but important binary signal.

It enables:

* Fast operational alerts
* Root cause prioritization
* Conditional escalation logic

It complements the percent metric.

You now have both:

* Severity measure
* Presence indicator

That’s good system design.

---

# 7️⃣ Strengths in This Implementation

✔ Store-level matching
✔ Clean normalization helpers
✔ Deterministic logic
✔ No hidden inference
✔ Efficient set membership
✔ Percentage-based attribution
✔ Binary flag for decision logic

This is clean and practical.

---

# 8️⃣ Important Refinements (High-Value Improvements)

Now we move to serious engineering precision.

---

## 🔹 1️⃣ Critical Modeling Nuance — Baseline vs Week-Level Baseline

Currently:

```python
if spend < baseline_revenue
```

You are comparing each week’s spend to the overall baseline average.

This may over-count minor fluctuations as “gap weeks.”

More precise alternative (future refinement):

Define gap weeks as:

```python
if spend < (baseline_revenue * gap_threshold)
```

Where `gap_threshold` might be 0.9 or 0.95.

That avoids flagging small dips.

For MVP this is fine.
But worth future tuning.

---

## 🔹 2️⃣ Store Consistency Assumption

Docstring says:

> Assumes sales_sorted has same store_id for customer.

But code does not enforce that.

If a customer shops multiple stores:

You will evaluate each week separately.

That may be fine — but it contradicts the comment.

Two options:

* Enforce store consistency
* Or update docstring to reflect reality

I recommend updating the docstring for accuracy.

---

## 🔹 3️⃣ Consider Restricting to Recent Window

Right now you use entire `sales_sorted`.

You may want:

Only evaluate last `structural_window_weeks`.

That aligns attribution with structural modeling.

Otherwise, old gap weeks may influence attribution.

---

## 🔹 4️⃣ Percent Precision

You round to 4 decimals.

Good for math.
But you may want to round to 2 in report layer only.

Minor detail.

---

# 9️⃣ Why This Design Is Enterprise-Aligned

This module:

* Assigns responsibility
* Quantifies operational friction
* Prevents misclassification of churn
* Enables targeted intervention
* Reduces cross-functional blame ambiguity

Most AI systems would:

* Blend operational and behavioral decline
* Leave attribution to humans
* Provide no accountability metrics

This one does not.

It explicitly models operational impact.

That is strategically strong.

---

# 🔥 Strategic Impact

This is the moment the orchestrator becomes:

Not just an exposure engine
but
an operational accountability engine.

This shifts the system from:

Analytics
to
Decision intelligence.

---

# 🏁 Final Assessment

This module is:

* Clean
* Efficient
* Architecturally sound
* Business-aligned
* Implementation-ready

With minor refinements, it is enterprise-grade.


In [ ]:
"""
Store-level stockout attribution: overlap of gap weeks with store stockout weeks (RGOv2_5).
"""

from typing import Any, Dict, List, Set, Tuple


def _week_str(row: Dict[str, Any]) -> str:
    """Normalize week from row (week_start_date or week_start)."""
    w = row.get("week_start_date") or row.get("week_start")
    if w is None:
        return ""
    if hasattr(w, "strftime"):
        return w.strftime("%Y-%m-%d")
    return str(w)[:10]


def _store_str(row: Dict[str, Any]) -> str:
    return str(row.get("store_id", ""))


def stockout_overlap_for_customer(
    sales_sorted: List[Dict[str, Any]],
    baseline_revenue: float,
    stockout_store_weeks: Set[Tuple[str, str]],
) -> Dict[str, Any]:
    """
    For one customer: gap_weeks = weeks where weekly_spend < baseline or 0.
    stockout_overlap_weeks = count of gap_weeks where (store_id, week) is in stockout_store_weeks.
    percent_gap_due_to_stockout = overlap / len(gap_weeks) if gap_weeks else 0.
    Assumes sales_sorted has same store_id for customer (use first row's store_id if mixed).
    """
    gap_weeks: List[Tuple[str, str]] = []
    for r in sales_sorted:
        store = _store_str(r)
        week = _week_str(r)
        if not week:
            continue
        spend = float(r.get("weekly_spend") or 0)
        if spend < baseline_revenue or spend == 0:
            gap_weeks.append((store, week))

    overlap = sum(1 for (s, w) in gap_weeks if (s, w) in stockout_store_weeks)
    total_gap_weeks = len(gap_weeks)
    if total_gap_weeks == 0:
        percent = 0.0
    else:
        percent = round(overlap / total_gap_weeks, 4)

    return {
        "stockout_overlap_weeks": overlap,
        "percent_gap_due_to_stockout": percent,
        "stockout_impact_flag": overlap >= 1,
    }
